In [ ]:
%pylab inline
import numpy as np
from sklearn import preprocessing
from keras.datasets import mnist
from keras.utils import np_utils
np.set_printoptions(suppress=True)
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
def preproc(X_train, y_train):
    X = X_train
    X = X.reshape(X.shape[0], X.shape[1] * X.shape[2]).astype("float32")
    #X = X/255
    X = preprocessing.scale(X)
    Y = np_utils.to_categorical(y_train, 10)
    return X, Y

X, Y = preproc(X_train, y_train)
Xt, Yt = preproc(X_test, y_test)

imshow(X[5].reshape((sqrt(X[1].shape),sqrt(X[1].shape))))

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop

# uniform init
np.random.seed(1337)

# 208896 params
model = Sequential()
model.add(Dense(784, 32))
model.add(Activation('tanh'))
model.add(Dense(32, 784))

# optimize
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd)

In [ ]:
# repro
model.fit(X, X, batch_size=128, nb_epoch=100)

In [ ]:
img = 5

# 784*32 + 32 + 32*784 + 784 = 50992
print W.shape, b.shape, Wp.shape, bp.shape
W,b,Wp,bp = map(lambda x: x.get_value(), model.params)

im = X[img].reshape((sqrt(X[1].shape),sqrt(X[1].shape)))
figure(), imshow(im)

x = X[img]
m = np.tanh(np.dot(x, W) + b)
y = np.dot(m, Wp) + bp

pred = y.reshape((sqrt(X[1].shape),sqrt(X[1].shape)))
figure(), imshow(pred)

print np.mean((im-pred)**2)

In [ ]:
model.evaluate(X, X, verbose=2)
model.evaluate(Xt, Xt, verbose=2)
pass

In [ ]:
import pickle

pickle.dump(map(lambda x: x.get_value(), model.params), open("sample/sgdweights", "wb"))